In [4]:
import pandas as pd
import datetime as dt

In [2]:
events = pd.read_csv('../fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv')

/home/aaron/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_data = pd.read_csv('../fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [6]:
events['week'] = pd.to_datetime(events['timestamp']).dt.week

In [7]:
events['week'].value_counts()

21    540477
20    495049
22    408822
19    166141
18    118139
17     96538
16     67727
15     65513
14     60947
13     55405
12     55377
11     40157
10     31867
9      28420
8      21058
5      16097
7      15546
6      15323
3      12688
2      12122
4      11537
1       6731
Name: week, dtype: int64

In [54]:
#SEPARO EN PERIODOS DE DOS SEMANAS
def periodo (week):
    per= 0
    if((week == 1)|(week == 2)):
        per = 1
    if((week == 3)|(week == 4)):
        per = 2
    if((week == 5)|(week == 6)):
        per = 3
    if((week == 7)|(week == 8)):
        per = 4
    if((week == 9)|(week == 10)):
        per = 5
    if((week == 11)|(week == 12)):
        per = 6
    if((week == 13)|(week == 14)):
        per = 7
    if((week == 15)|(week == 16)):
        per = 8
    if((week == 17)|(week == 18)):
        per = 9
    if((week == 19)|(week == 20)):
        per = 10    
    if((week == 21)|(week==22)):
        per = 11
    return per

events['periodo'] = events['week'].apply(lambda week: periodo(week))

In [55]:
events['periodo'].value_counts()

11    949299
10    661190
9     214677
8     133240
7     116352
6      95534
5      60287
4      36604
3      31420
2      24225
1      18853
Name: periodo, dtype: int64

# Feature: Cantidad de eventos por usuario 

Solo trabajo con alguno de los periodos

In [20]:
cantidad_eventos10 = events10[['person','event','timestamp']]
cantidad_eventos10 = cantidad_eventos1.groupby(['person','event']).count()
cantidad_eventos10 = cantidad_eventos10.unstack().reset_index()

In [22]:
cantidad_eventos10.fillna(value=0,inplace=True)